# Alternative Fuel Stations

In [11]:
import requests
import pandas as pd
from config import api_key

# Construct the API URL
url = f"https://developer.nrel.gov/api/alt-fuel-stations/v1.json?api_key={api_key}"

# Make the API request
response = requests.get(url)

# Check for successful response
if response.status_code == 200:
    try:
        # Parse the JSON response
        data = response.json()
        
        # Extract fuel stations data
        fuel_stations = data["fuel_stations"]
        
        # Define columns to keep
        columns_to_keep = [
            "id", "fuel_type_code", "station_name", "street_address", 
            "city", "state", "zip", "country", "status_code", "expected_date", 
            "access_code", "owner_type_code", "federal_agency", 
            "ev_level1_evse_num", "ev_level2_evse_num", "ev_dc_fast_num", 
            "ev_other_evse", "ev_connector_types", "ev_network", "ev_pricing", 
            "ev_renewable_source", "latitude", "longitude", "open_date", 
            "date_last_confirmed", "facility_type", "restricted_access"
        ]
        
        # Create a DataFrame with selected columns
        df = pd.DataFrame(fuel_stations)[columns_to_keep]
        
    except json.JSONDecodeError as e:
        print("Error decoding JSON:", e)
else:
    print("API request failed with status code:", response.status_code)

In [12]:
df.head()

,id,fuel_type_code,station_name,street_address,city,state,zip,country,status_code,expected_date,...,ev_connector_types,ev_network,ev_pricing,ev_renewable_source,latitude,longitude,open_date,date_last_confirmed,facility_type,restricted_access
0,17,CNG,Spire - Montgomery Operations Center,2951 Chestnut St,Montgomery,AL,36107,US,E,None,...,None,None,None,None,32.367916,-86.267021,2010-12-01,2023-04-06,STANDALONE_STATION,None
1,45,CNG,Metropolitan Atlanta Rapid Transit Authority,2424 Piedmont Rd NE,Atlanta,GA,30324,US,E,None,...,None,None,None,None,33.821911,-84.367461,1996-12-15,2023-01-10,FLEET_GARAGE,None
2,64,CNG,United Parcel Service,270 Marvin Miller Dr,Atlanta,GA,30336,US,E,None,...,None,None,None,None,33.760256,-84.543822,1997-01-01,2022-06-14,STANDALONE_STATION,None
3,73,CNG,Arkansas Oklahoma Gas Corp,2100 S Waldron Rd,Fort Smith,AR,72903,US,E,None,...,None,None,None,None,35.362213,-94.375338,1997-01-01,2023-02-13,UTILITY,False
4,81,CNG,Clean Energy - Logan International Airport,1000 Cottage St Ext,East Boston,MA,02128,US,E,None,...,None,None,None,None,42.374706,-71.026549,1996-11-15,2022-12-13,AIRPORT,False


In [13]:
# Export DataFrame to CSV
csv_filename = "fuel_stations.csv"
df.to_csv(csv_filename, index=False)
print(f"DataFrame exported to {csv_filename}")

DataFrame exported to fuel_stations.csv


In [14]:
# Export DataFrame to JSON
json_filename = "fuel_stations.json"
df.to_json(json_filename, orient="records")
print(f"DataFrame exported to {json_filename}")

DataFrame exported to fuel_stations.json


## Cleaning/Preparing Data for Analysis

The exported CSV file above was imported into MongoDB. This section will focus on cleaning the data and preparing it for analysis.

In [15]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [16]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [17]:
# confirm that our new database "alt_fuel" was created
print(mongo.list_database_names())

['admin', 'alt_fuel', 'classDB', 'config', 'gardenDB', 'local', 'met', 'test', 'travel_db', 'uk_food']


In [20]:
# assign the alt_fuel database to a variable name
db = mongo['alt_fuel']

In [21]:
# review the collections in our new database
print(db.list_collection_names())

['fuel_stations']


In [23]:
# review a document in our establishment collection.
pprint(db.fuel_stations.find_one())

{'_id': ObjectId('64ce71f5e4c0a798bc65120a'),
 'access_code': 'public',
 'city': 'Fort Smith',
 'country': 'US',
 'date_last_confirmed': '2023-02-13',
 'ev_connector_types': '',
 'ev_dc_fast_num': '',
 'ev_level1_evse_num': '',
 'ev_level2_evse_num': '',
 'ev_network': '',
 'ev_other_evse': '',
 'ev_pricing': '',
 'ev_renewable_source': '',
 'expected_date': '',
 'facility_type': 'UTILITY',
 'federal_agency': '',
 'fuel_type_code': 'CNG',
 'id': 73,
 'latitude': 35.362213,
 'longitude': -94.375338,
 'open_date': '1997-01-01',
 'owner_type_code': 'T',
 'restricted_access': 'False',
 'state': 'AR',
 'station_name': 'Arkansas Oklahoma Gas Corp',
 'status_code': 'E',
 'street_address': '2100 S Waldron Rd',
 'zip': 72903}


In [24]:
# assign the collection to a variable
stations = db['fuel_stations']

In [31]:
# drop the zip code field - won't be needed for our analysis
stations.update_many(
   { },
   { "$unset": { "zip": "" } }
)

# review a document in our establishment collection to see if the "zip" field was dropped
pprint(stations.find_one())

{'_id': ObjectId('64ce71f5e4c0a798bc65120a'),
 'access_code': 'public',
 'city': 'Fort Smith',
 'country': 'US',
 'date_last_confirmed': '2023-02-13',
 'ev_connector_types': '',
 'ev_dc_fast_num': '',
 'ev_level1_evse_num': '',
 'ev_level2_evse_num': '',
 'ev_network': '',
 'ev_other_evse': '',
 'ev_pricing': '',
 'ev_renewable_source': '',
 'expected_date': '',
 'facility_type': 'UTILITY',
 'federal_agency': '',
 'fuel_type_code': 'CNG',
 'id': 73,
 'latitude': 35.362213,
 'longitude': -94.375338,
 'open_date': '1997-01-01',
 'owner_type_code': 'T',
 'restricted_access': 'False',
 'state': 'AR',
 'station_name': 'Arkansas Oklahoma Gas Corp',
 'status_code': 'E',
 'street_address': '2100 S Waldron Rd'}


In [36]:
# Delete all documents where 'state' is not a US state such as PR, BC, QC, & ON
stations.delete_many({"state" : "PR"})
stations.delete_many({"state" : "BC"})
stations.delete_many({"state" : "QC"})
stations.delete_many({"state" : "ON"})

# review a document in our establishment collection to see if the "zip" field was dropped
stations.count_documents({})

73834

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [ ]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [ ]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

In [ ]:
# Check that the new restaurant was inserted
establishments.find_one({"BusinessName" : "Penang Flavours"})

###or use this one below
#Filter results by name
query = {'BusinessName': 'Penang Flavours'}
#results = establishments.find(query)
#for result in results:
    #pprint(result)
establishments.find_one(query)

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [ ]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType': "Restaurant/Cafe/Canteen"}
fields = {"BusinessTypeID": 1, "BusinessType" : 1}
results = establishments.find(query,fields)
for result in results:
    pprint(result)

In [ ]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
#query = {'BusinessType': "Restaurant/Cafe/Canteen"}
#fields = ["BusinessTypeID", "BusinessType"]
#establishments.find_one(query,fields)

3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one(new_restaurant,{"$set": {"BusinessTypeID": 1}})

In [ ]:
# Confirm that the new restaurant was updated
query = {"BusinessName" : "Penang Flavours"}
establishments.find_one(query)

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"
establishments.count_documents({"LocalAuthorityName": "Dover"})

In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({"LocalAuthorityName" : "Dover"})

In [ ]:
# Check if any remaining documents include Dover
pprint(establishments.count_documents({"LocalAuhtorityName" : "Dover"}))

In [ ]:
# Check that other documents remain with 'find_one'
establishments.find_one({})

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({}, [{"$set" : {"geocode.longitude" : {"$toDouble" : "$geocode.longitude"},
                                         "geocode.latitude" : {"$toDouble" : "$geocode.latitude"}}}])

Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}}])

In [ ]:
# Change the data type from String to Integer for RatingValue
#establishments.update_many({}, [{"$set" : {"RatingValue" : {"$toInt" : "RatingValue"}}}])
establishments.update_many({}, [{"$set" : {"RatingValue" : {"$toInt" : "$RatingValue"}}}])
#establishments.update_many({"RatingValue": {"$in" : "RatingValue"}}, [{"$set" : {"$toInt" : "RatingValue"}}])
#establishments.update_many({"RatingValue": [{'$set':{"RatingValue" : {"$toInt" : "RatingValue"}}}])

In [ ]:
# Check that the coordinates and rating value are now numbers
# Change the data type from String to Double for CO.PERCENT_COMPLETE 
# and from String to Int for CO.DAILY_AQI_VALUE
# in the ohio_daily_records collection
query = {}
fields = {"geocode.longitude" : 1, "geocode.latitude" : 1}
list(establishments.find(query,fields))[0:5]